In [35]:
from time import sleep, time
import ast
from io import StringIO
import inspect
import re

In [36]:
def sleep_long():
    sleep(5) 

In [37]:
def sleep_short():
    sleep(1)

In [38]:
def my_func():
    sleep_long()
    sleep_short()
    return None

## Lets use the AST to easily create our benchmarking function

#### Getting our function as a string

In [39]:
func = inspect.getsource(my_func)

In [40]:
func

'def my_func():\n    sleep_long()\n    sleep_short()\n    return None\n'

In [41]:
tree = ast.parse(StringIO(func).read())

In [42]:
ast.dump(tree.body[0].body[0])

"Expr(value=Call(func=Name(id='sleep_long', ctx=Load()), args=[], keywords=[]))"

#### Getting information about each of the first braches and creating a new first layer of branches

In [43]:
def parse_dump(string):
    '''
    Gets information from the AST dump and return the AST object and the id
    '''
    re1 = "id='(.*?)',"
    return string.split('(')[0] + ' - ' + re.search(re1, string).group(1)

In [44]:
performance = dict()
t = 0
new_first_layer = list()
for node in tree.body[0].body:
    if node.__class__ != ast.Return:
        node_dump = ast.dump(node)
        parsed_dump = parse_dump(node_dump)
        performance[parsed_dump] = list()
        start_time = ast.parse(f"t{t} = time()").body[0]
        t+=1
        end_time = ast.parse(f"t{t} = time()").body[0]
        dif = ast.parse(f"performance['{parsed_dump}'].append(t{t}-t{t-1})").body[0]
        new_first_layer.extend([start_time, node, end_time, dif])
        t+=1
    else:
        new_first_layer.extend([node])

In [45]:
performance

{'Expr - sleep_long': [], 'Expr - sleep_short': []}

In [46]:
tree.body[0].body = new_first_layer

#### Creating function as benchmark

In [47]:
tree.body[0].name = f'benchmark_{tree.body[0].name}'

In [48]:
tree.body[0].body

In [49]:
exec(compile(tree, filename="<ast>", mode="exec"))

In [50]:
benchmark_my_func()

In [51]:
performance

{'Expr - sleep_long': [5.004082918167114],
 'Expr - sleep_short': [1.0002343654632568]}

## Let's benchmark

In [52]:
for i in range(0,5):
    benchmark_my_func()

In [53]:
performance

{'Expr - sleep_long': [5.004082918167114,
  5.003403186798096,
  5.003105163574219,
  5.0036091804504395,
  5.003881931304932,
  5.000932693481445],
 'Expr - sleep_short': [1.0002343654632568,
  1.0043470859527588,
  1.002871036529541,
  1.0050911903381348,
  1.0010039806365967,
  1.0013761520385742]}

## Lets create a function to accept any function

In [54]:
def create_benchmark_func(tree, performance):
    t = 0
    new_first_layer = list()
    for node in tree.body[0].body:
        if node.__class__ != ast.Return:
            node_dump = ast.dump(node)
            parsed_dump = parse_dump(node_dump)
            performance[parsed_dump] = list()
            start_time = ast.parse(f"t{t} = time()").body[0]
            t+=1
            end_time = ast.parse(f"t{t} = time()").body[0]
            dif = ast.parse(f"performance['{parsed_dump}'].append(t{t}-t{t-1})").body[0]
            new_first_layer.extend([start_time, node, end_time, dif])
            t+=1
        else:
            new_first_layer.extend([node]) 
    tree.body[0].body = new_first_layer
    tree.body[0].name = f'benchmark_{tree.body[0].name}'
    return tree

In [55]:
def benchmark(func, iteration, *args, **kwargs):
    func_str = inspect.getsource(func)
    tree = ast.parse(StringIO(func_str).read())
    global performance
    performance = dict()
    exec(compile(create_benchmark_func(tree, performance), filename="<ast>", mode="exec"))
    for i in range(iteration):
        args_str = ', '.join(args)
        kwargs_str = ', '.join('%s=%r' % x for x in kwargs.items())
        eval(f'benchmark_{func.__name__}({args_str+kwargs_str})')
    return performance

In [56]:
benchmark(my_func, 1)

{'Expr - sleep_long': [5.003541946411133],
 'Expr - sleep_short': [1.0012168884277344]}

In [57]:
def hello(string):
    print(f'hello_{string}')

In [59]:
benchmark(hello, 5, "'world'")

hello_world
hello_world
hello_world
hello_world
hello_world


{'Expr - print': [0.00010776519775390625,
  4.601478576660156e-05,
  4.506111145019531e-05,
  4.506111145019531e-05,
  4.410743713378906e-05]}